In [119]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


        
#import seaborn as sns
#import matplotlib.pyplot as plt
#from matplotlib.lines import Line2D
import plotly as py
import plotly.express as px
#import plotly.graph_objects as go

/kaggle/input/farmers-markets-in-the-united-states/farmers_markets_from_usda.csv
/kaggle/input/farmers-markets-in-the-united-states/wiki_county_info.csv


In [120]:
# Reading in the CSV data for farmers markets and U.S. counties
us_farmers_market_df = pd.read_csv("../input/farmers-markets-in-the-united-states/farmers_markets_from_usda.csv")
counties_df = pd.read_csv("../input/farmers-markets-in-the-united-states/wiki_county_info.csv")

In [121]:
us_farmers_market_df[us_farmers_market_df['x'].isnull()]
counties_df[counties_df['county'].isnull()]

,number,county,State,per capita income,median household income,median family income,population,number of households
96,NaN,NaN,Maryland,"$36,354","$73,538","$88,738","5,834,299","2,146,240"
105,NaN,NaN,New Jersey,"$36,027","$71,629","$87,347","8,832,406","3,186,418"
2205,30%,NaN,NaN,"$20,530",NaN,NaN,NaN,NaN
2522,20%,NaN,NaN,"$19,280",NaN,NaN,NaN,NaN
3152,NaN,NaN,Puerto Rico,"$12,081","$19,775","$23,793","3,195,153","1,222,606"


In [122]:
# Dropping US farmers markets that did not provide a fixed coordinates.
us_farmers_market_df = us_farmers_market_df[us_farmers_market_df['x'].notna()]

# Dropping counties withtheir name as null as these are the error entries
counties_df = counties_df[counties_df['county'].notna()]

# Removing the $ sign from the dataframe
counties_df[counties_df.columns[3:]] = counties_df[us_counties_df.columns[3:]].replace('[\$,]', '', regex=True).astype(float)

In [123]:
us_farmers_market_df2 = us_farmers_market_df.copy()
us_farmers_market_df2['County-StateM']=us_farmers_market_df2['County'].astype(str) + ',' + us_farmers_market_df2['State'].astype(str)

counties_df2 = counties_df.copy()
counties_df2['County-State']=counties_df2['county'].astype(str) + ',' + counties_df2['State'].astype(str)

# Count the number of farmers markets by State
us_farmers_market_state_count2 = us_farmers_market_df2.groupby('County-StateM').count()
us_farmers_market_state_count2['County-State'] = us_farmers_market_state_count2.index.values 

merged_county_df = pd.merge(left=us_counties_df2, right=us_farmers_market_state_count2, left_on='County-State', right_on='County-State')

merged_county_df = merged_county_df.rename(columns={"State_x": "state", "FMID": "Farmers Market Count"})

# Creating a clean merged dataframe with only the relevant columns
clean_merged_county_df = merged_county_df[['County-State','state','per capita income'
                                           #,'median household income','median family income'
                                           ,'population'
                                           #,'number of households'
                                           ,'Farmers Market Count']].copy()
#clean_merged_county_df['Household Density']=(clean_merged_county_df['population'].div(clean_merged_county_df['number of households']))


clean_merged_county_df = clean_merged_county_df.rename(columns={"per capita income": "Per Capita Income",
                                                                #, "median household income": "Median Household Income", "median family income": "Median Family Income", 
                                                                "population": "Population",
                                                                #, "number of households":"Number of Households",
                                                                "state":"State"})

clean_merged_county_df['Farmers Market 100k Density']=clean_merged_county_df['Farmers Market Count'].div(clean_merged_county_df['Population'])*100000

clean_merged_county_df.head()

,County-State,State,Per Capita Income,Population,Farmers Market Count,Farmers Market 100k Density
0,"Arlington,Virginia",Virginia,62018.0,214861.0,12,5.585006
1,"Marin,California",California,56791.0,254643.0,15,5.890600
2,"Pitkin,Colorado",Colorado,51814.0,17173.0,1,5.823094
3,"Los Alamos,New Mexico",New Mexico,51044.0,17979.0,1,5.562045
4,"Hunterdon,New Jersey",New Jersey,50349.0,127047.0,5,3.935551


In [124]:
# Scatter plots of income and farmers market relationships
fig = px.scatter(clean_merged_county_df, x="Per Capita Income", y="Farmers Market 100k Density",
                 #color="County-State",
                 hover_data=['County-State'],size_max=50, trendline="ols")

fig.show()